In [63]:
import redis
from mlrepricer import redisdb, helper, parser, setup, updateprice
import xmltodict

In [64]:
helper.rediscred

{'host': '127.0.0.1', 'port': '6379'}

In [65]:
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
redisdb.server_start()

In [42]:
m = r.scan_iter(match=r'[a-zA-Z0-9]'*10)

In [76]:
r.srem()

TypeError: srem() missing 1 required positional argument: 'name'

In [27]:
m[1][0]['Body']

TypeError: tuple indices must be integers or slices, not str

In [45]:
# example get value by key for 
# type: specified members with the specified scores to the sorted set stored at key
m = r.zscan(next(m))

In [67]:
# m1 is where all messages are, the next 0 is where the latest message is stored (highest timestamp value)
# the next 0 is getting the stored data a string which is the message how we get it from the aws queue
x = m[1][0][0]
xmltodict.parse(eval(x)[0]['Body'])
#eval(x)[0]['ReceiptHandle']

OrderedDict([('Notification',
              OrderedDict([('NotificationMetaData',
                            OrderedDict([('NotificationType',
                                          'AnyOfferChanged'),
                                         ('PayloadVersion', '1.0'),
                                         ('UniqueId',
                                          '1ff117ca-a994-4f9d-a5a0-fba66adabc05'),
                                         ('PublishTime',
                                          '2019-01-05T04:35:31.005Z'),
                                         ('SellerId', 'A2EHWGAW6J9W8Q'),
                                         ('MarketplaceId',
                                          'A1PA6795UKMFR9')])),
                           ('NotificationPayload',
                            OrderedDict([('AnyOfferChangedNotification',
                                          OrderedDict([('OfferChangeTrigger',
                                                        Ordered

Example iterate over all keys

In [11]:
def asin_generator():
    """
    Match keys which contain all messages per asin.
    Key example: B07B9NGM2J
    Return a generator of keys you can use to get the actual messages.
    """
    return r.scan_iter(match=r'[a-zA-Z0-9]'*10)

In [14]:
def actionable_asins():
    """
    Get asin we have to take actions.
    one key: updated_asins
    """
    return r.sscan_iter('updated_asins')

In [15]:
def parsed_messages(asin):
    """Make a list with all messages parsed."""
    mlist = r.zrevrangebyscore(asin, 'inf', '-inf')
    return [parser.parse(eval(message)) for message in mlist]

In [16]:
#[x for x in message_key()]

In [17]:
result = []
asin_gen = asin_generator()
for asin in asin_gen:
    result.append(parsed_messages(asin))

In [18]:
result[33][1]

IndexError: list index out of range

In [19]:
updateprice.get_latest_message(asin)

NameError: name 'asin' is not defined